In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy
from sklearn.metrics import f1_score,precision_score,recall_score
import os

In [3]:
import json

dev_path = "E:/Professional Masters in Data Science/SEM 3/NLP/Project/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/training_data/dev.json"
with open(dev_path) as json_file:
    dev = json.load(json_file)

# Example instance
print(dev[list(dev.keys())[1]])

{'Type': 'Comparison', 'Section_id': 'Eligibility', 'Primary_id': 'NCT00425854', 'Secondary_id': 'NCT01224678', 'Statement': 'Patients with significantly elevated ejection fraction are excluded from the primary trial, but can still be eligible for the secondary trial if they are 55 years of age or over', 'Label': 'Contradiction'}


In [20]:
uuid_list = list(dev.keys())
statements = []
gold_dev_primary_evidence = []
gold_dev_secondary_evidence = []
for i in range(len(uuid_list)):
  #Retrieve all statements from the development set
  statements.append(dev[uuid_list[i]]["Statement"])

In [21]:
statements

['there is a 13.2% difference between the results from the two the primary trial cohorts',
 'Patients with significantly elevated ejection fraction are excluded from the primary trial, but can still be eligible for the secondary trial if they are 55 years of age or over',
 'a significant number of the participants in the secondary trial and the primary trial suffered from Enterocolitis',
 'the primary trial does not report the PFS or objective response rate of its patient cohort',
 'Prior treatment with fulvestrant or with a phosphatidylinositol 3-kinase (PI3K) inhibitor will not prevent a patient from participating in the primary trial',
 'only patients with a HER2-positive status can take part in the primary trial',
 'the shortest PFS in cohort 1 of the primary trial was under 7 months, in cohort 2 the shortest was just over 7 months',
 'Both cohorts of the primary trial undergo a total of 17 cycles, each lasting 3 weeks .',
 'the outcome measurement of the primary trial is The lengt

In [13]:
Results = {}
path="E:/Professional Masters in Data Science/SEM 3/NLP/Project/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/training_data/CT json"
for i in range(len(uuid_list)):
  primary_ctr_path = os.path.join(path,dev[uuid_list[i]]["Primary_id"]+".json")
  with open(primary_ctr_path) as json_file:
    primary_ctr = json.load(json_file)

  #Retrieve the full section from the primary trial
  primary_section = primary_ctr[dev[uuid_list[i]]["Section_id"]]

  #Convert a primary section entries to a matrix of TF-IDF features.
  vectorizer = TfidfVectorizer().fit(primary_section)
  X_s = vectorizer.transform([statements[i]])
  X_p = vectorizer.transform(primary_section)
  #Compute the cosine similarity between the primary section entries and the statement
  primary_scores = cosine_distances(X_s, X_p)
  #Repeat for the secondary trial
  if dev[uuid_list[i]]["Type"] == "Comparison":
    secondary_ctr_path = os.path.join(path,dev[uuid_list[i]]["Secondary_id"]+".json")
    with open(secondary_ctr_path) as json_file:
      secondary_ctr = json.load(json_file)
    secondary_section = secondary_ctr[dev[uuid_list[i]]["Section_id"]]
    vectorizer = TfidfVectorizer().fit(secondary_section)
    X_s = vectorizer.transform([statements[i]])
    X_p = vectorizer.transform(secondary_section)
    secondary_scores = cosine_distances(X_s, X_p)
    #Combine and average the cosine distances of all entries from the relevant section of the primary and secondary trial
    combined_scores = []
    combined_scores.extend(secondary_scores[0])
    combined_scores.extend(primary_scores[0])
    score = numpy.average(combined_scores)
    #If the cosine distance is gless than 0.9 the prediction is entailment
    if score > 0.9:
      Prediction = "Contradiction"
    else:
      Prediction = "Entailment"
    Results[str(uuid_list[i])] = {"Prediction":Prediction}
  else:
    #If the cosine distance is greater than 0.9 the prediction is contradiction
    score = numpy.average(primary_scores)
    if score > 0.9:
      Prediction = "Contradiction"
    else:
      Prediction = "Entailment"
    Results[str(uuid_list[i])] = {"Prediction":Prediction}

    

In [15]:
print(Results)
with open("/content/drive/MyDrive/Task-2-SemEval-2024/results.json",'w') as jsonFile:
    jsonFile.write(json.dumps(Results,indent=4))

{'1adc970c-d433-44d0-aa09-d3834986f7a2': {'Prediction': 'Entailment'}, '6b9162d0-0816-46d4-81af-c60028dcc63b': {'Prediction': 'Entailment'}, '0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f': {'Prediction': 'Entailment'}, 'cc1f712a-2116-4e40-9810-f315e3fa5ff8': {'Prediction': 'Entailment'}, '904061c0-14fa-4f13-9118-9a41e24fa8eb': {'Prediction': 'Entailment'}, '43ee7645-ce1e-42d5-9a74-3e379f6f367b': {'Prediction': 'Entailment'}, '0cef8c8e-7986-46c7-a597-c5733a9899c0': {'Prediction': 'Entailment'}, '43ce26e5-03fa-4e9d-b0eb-6ea356295753': {'Prediction': 'Entailment'}, '3facad41-0221-42f8-834d-470e65c4aad5': {'Prediction': 'Entailment'}, '9cbc00e9-3a2d-4471-a93e-72c95132fb6a': {'Prediction': 'Entailment'}, '8b91cab9-d858-45f3-bf8d-3d6fc55b4818': {'Prediction': 'Entailment'}, '4a75574c-fa86-4e62-a210-81c7b98a3807': {'Prediction': 'Entailment'}, 'd0b50aeb-aad8-4a8d-aae6-5c58a7d382c7': {'Prediction': 'Entailment'}, 'b0b61978-57db-4a1c-812c-509e8b05f2dc': {'Prediction': 'Entailment'}, '24b85b44-b9e6-4c28

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Task-2-SemEval-2024/results.json'

In [16]:
def main():

    gold = dev
    results = Results
    uuid_list = list(results.keys())

    results_pred = []
    gold_labels = []
    for i in range(len(uuid_list)):
        if results[uuid_list[i]]["Prediction"] == "Entailment":
            results_pred.append(1)
        else:
            results_pred.append(0)
        if gold[uuid_list[i]]["Label"] == "Entailment":
            gold_labels.append(1)
        else:
            gold_labels.append(0)

    f_score = f1_score(gold_labels,results_pred)
    p_score = precision_score(gold_labels,results_pred)
    r_score = recall_score(gold_labels,results_pred)

    print('F1:{:f}'.format(f_score))
    print('precision_score:{:f}'.format(p_score))
    print('recall_score:{:f}'.format(r_score))

if '__main__' == __name__:
    main()

F1:0.666667
precision_score:0.500000
recall_score:1.000000


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import numpy as np

Results = {}

for i in range(len(uuid_list)):
    primary_ctr_path = os.path.join(path, dev[uuid_list[i]]["Primary_id"] + ".json")
    
    with open(primary_ctr_path) as json_file:
        primary_ctr = json.load(json_file)
    
    primary_section = primary_ctr[dev[uuid_list[i]]["Section_id"]]
    
    # Fit the vectorizer on all sections in the training data
    vectorizer = TfidfVectorizer().fit(primary_section + secondary_section)
    
    X_s = vectorizer.transform([statements[i]])
    X_p_primary = vectorizer.transform(primary_section)
    
    primary_scores = cosine_similarity(X_s, X_p_primary)[0]
    
    if dev[uuid_list[i]]["Type"] == "Comparison":
        secondary_ctr_path = os.path.join(path, dev[uuid_list[i]]["Secondary_id"] + ".json")
        
        with open(secondary_ctr_path) as json_file:
            secondary_ctr = json.load(json_file)
        
        secondary_section = secondary_ctr[dev[uuid_list[i]]["Section_id"]]
        
        X_p_secondary = vectorizer.transform(secondary_section)
        secondary_scores = cosine_similarity(X_s, X_p_secondary)[0]
        
        # Combine and average the cosine distances
        combined_scores = np.concatenate([secondary_scores, primary_scores])
        score = np.average(combined_scores)
    else:
        score = np.average(primary_scores)

    # Adjust the threshold based on your experiments
    if score > 0.8:
        Prediction = "Contradiction"
    else:
        Prediction = "Entailment"
    
    Results[str(uuid_list[i])] = {"Prediction": Prediction}


In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize variables to store the best metrics and threshold
best_precision = 0
best_recall = 0
best_f1 = 0
best_threshold = 0

# Experiment with different threshold values
for threshold in np.arange(0.1, 1.0, 0.1):
    # Use the threshold in your existing code
    if score > threshold:
        Prediction = "Contradiction"
    else:
        Prediction = "Entailment"
    
    # Compute precision, recall, and F1
        gold = dev
    results = Results
    uuid_list = list(results.keys())

    results_pred = []
    gold_labels = []
    for i in range(len(uuid_list)):
        if results[uuid_list[i]]["Prediction"] == "Entailment":
            results_pred.append(1)
        else:
            results_pred.append(0)
        if gold[uuid_list[i]]["Label"] == "Entailment":
            gold_labels.append(1)
        else:
            gold_labels.append(0)
            
    y_true = gold_labels
    y_pred = results_pred
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Update the best metrics and threshold if necessary
    if f1 > best_f1:
        best_f1 = f1
        best_precision = precision
        best_recall = recall
        best_threshold = threshold

# Print the best metrics and threshold
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)
print("Best F1:", best_f1)
print("Best Threshold:", best_threshold)


Best Precision: 0.5
Best Recall: 1.0
Best F1: 0.6666666666666666
Best Threshold: 0.1
